Helper, prototype script to convert Keras networks using FKB


In [1]:
# This cell is necessary to give access to the Fortran Keras Bridge on the path.
# Another option is copying this notebook into the FKB main folder.
import sys
PATH_FKB = '/work/bd1179/b309198/causal_discovery/FKB'
sys.path.append(PATH_FKB)


In [2]:
from pathlib import Path
from KerasWeightsProcessing.convert_weights import h5_to_txt

MODELS_FOLDER = "/work/bd1179/b309172/analysis/usmile/causality_convection/python/tuxedo_rasp-et-al/CBRAIN-CAM/saved_models/single_nn_3col/"
TARGET_FOLDER = "/work/bd1179/b309198/causal_discovery/fortran_models"

o_folder = Path(MODELS_FOLDER)


In [3]:
fortran_models = list()

keras_models = list(o_folder.iterdir())
for model_folder in keras_models:
    for h5_file in model_folder.glob("model.h5"):
        t_folder = Path(TARGET_FOLDER, model_folder.name)
        t_folder.mkdir(parents=True, exist_ok=True)
        
        t_name = h5_file.name.replace(".h5", ".txt")
        target = Path(t_folder, t_name)
        
        h5_to_txt(
            weights_file_name = h5_file,
            output_file_name = target
        )
        
        fortran_models.append(target)


/work/bd1179/b309198/causal_discovery/FKB/KerasWeightsProcessing/convert_weights.py:209: UserWarning: Unsupported layer, inputlayer, found! Skipping...
  warnings.warn(warning_str)


In [4]:
# Can't be compiled from Jupyter (can't module load gcc)

from tensorflow.keras.models import load_model
import numpy as np

inputs = [(i+1)%10 for i in range(94)]
for i, f90_model in enumerate(fortran_models):
    k_model = Path(keras_models[i], "model.h5")
    assert k_model.parent.name == f90_model.parent.name, \
            f'{k_model.parent.name} != {f90_model.parent.name}'
    !echo {f90_model}
    f90_prediction = !{PATH_FKB}/build/bin/./test_keras {f90_model}
    model = load_model(k_model)
    k_prediction = model.predict(np.array([inputs]))[0,0]
    print(f"{f90_prediction} == {k_prediction}")


/work/bd1179/b309198/causal_discovery/fortran_models/FLNT/model.txt
['   24.3603973    '] == 24.360397338867188
/work/bd1179/b309198/causal_discovery/fortran_models/TPHYSTND_524/model.txt
['   1.93219268    '] == 1.9321973323822021
/work/bd1179/b309198/causal_discovery/fortran_models/TPHYSTND_322/model.txt
['   10.6933279    '] == 10.693325996398926
/work/bd1179/b309198/causal_discovery/fortran_models/TPHYSTND_7/model.txt
['  -1.63952816    '] == -1.6395277976989746
/work/bd1179/b309198/causal_discovery/fortran_models/TPHYSTND_14/model.txt
['  0.447831243    '] == 0.4478313624858856
/work/bd1179/b309198/causal_discovery/fortran_models/TPHYSTND_54/model.txt
[' -0.564934194    '] == -0.5649340152740479
/work/bd1179/b309198/causal_discovery/fortran_models/PHQ_142/model.txt
['  -1.39496231    '] == -1.394962191581726
/work/bd1179/b309198/causal_discovery/fortran_models/PHQ_7/model.txt
[' -0.484897316    '] == -0.4848969578742981
/work/bd1179/b309198/causal_discovery/fortran_models/TPHYSTND